In [ ]:
#r "nuget: ExcelDataReader, 3.6.0"
#r "nuget: ExcelDataReader.DataSet, 3.6.0"

Installed Packages ExcelDataReader, 3.6.0 ExcelDataReader.DataSet, 3.6.0

In [ ]:
#r "nuget: ManuscriptsProcessor, 0.4.0.1"

Installed Packages ManuscriptsProcessor, 0.4.0.1

In [ ]:
using ExcelDataReader;

In [ ]:
using System.IO;

In [ ]:
using Newtonsoft.Json;

In [ ]:
using CorpusDraftCSharp;

In [ ]:
using System.Data;

In [ ]:
string fileDirPath = Path.Combine(Directory.GetCurrentDirectory(), "files");

In [ ]:
string filename = "new@HR-DAZD-336.xlsx";

In [ ]:
System.Text.Encoding.RegisterProvider(System.Text.CodePagesEncodingProvider.Instance);

In [ ]:
System.Data.DataSet ds = new();

using (var stream = File.Open(Path.Combine(fileDirPath, filename), FileMode.Open, FileAccess.Read))
{
    IExcelDataReader reader;
    reader = ExcelDataReader.ExcelReaderFactory.CreateReader(stream);
    var conf = new ExcelDataSetConfiguration()
    {
        ConfigureDataTable = (tableReader) => new ExcelDataTableConfiguration()
        {
            UseHeaderRow = true
        }
    };

    ds = reader.AsDataSet(conf);
    ds.Tables[0].Head(5, true);
    
}

1 mi Noi mi zam. ličn. — — — mn. 1. l. — — — N  
2 knez Conte knez im. — — — — jd. — — — m. N  
3 zadarski di Zara zadarski prid. — — — — jd. — — — m. N  
4 šuci giudice sudac im. — — — — mn. — — — m. N  
5 od di od prij. — — — — — — — — — —  


In [ ]:
var documentName = "Указ князя Анджело Барбаро судьям Бибине и Боканяца";
var googleDocPath = "_";
var filePath = "_";
var fields = new Dictionary<string, string>();
fields["Тип памятника"] = "Рукопись";
fields["Объем"] = "1 лист";
fields["Материал"] = "бумага";
fields["Датировка"] = "1727 г.";
fields["Место создания"] = "Задар";
fields["Место хранения"] = "Государственный архив Задара";
fields["Инвентарный номер"] = "HR-DAZD-336";

In [ ]:
var doc = new Document("0", documentName, filePath, googleDocPath);
doc.documentMetaData = new();
doc.documentMetaData.Add(new Dictionary<string, List<Value>>());


In [ ]:
foreach (KeyValuePair<string, string> kv in fields) {
    doc.documentMetaData[0][kv.Key] = new List<Value> {new Value(kv.Value)};
}

In [ ]:
Console.WriteLine(doc.documentMetaData[0]["Тип памятника"][0].name);

Рукопись


In [ ]:

    foreach (DataRow r in ds.Tables[0].Rows) {
        Console.WriteLine(r["Id"]);
    }

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
